---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [2]:
import pandas as pd
import numpy as np

## Cleaning Energy Data Source

# Assignment Questions 

In [173]:
def answer_one():
    # First let's load the data from the energy xlsx
    energy = pd.read_excel("Energy Indicators.xls", header=None, footer=None)
    energy.drop(energy.columns[:2], axis=1, inplace=True)
    energy = energy[11:245]
    energy.rename(columns={2 : "Country", 3 : 'Energy Supply', 4 : 'Energy Supply per Capita', 5 : '% Renewable'}, inplace=True)
    energy = energy[pd.isnull(energy['Country']) == False]
    # Replace Null values
    energy = energy.replace("...", np.nan)
    # Rescaling the energy supply column
    energy['Energy Supply'] = energy['Energy Supply'] * 1000000
    # Replacing country names
    #energy[energy['Country'] == 'Republic of Korea'].Country = 'South Korea'
    #energy[energy['Country'] == 'United States of America20'].Country = 'United States'
    #energy[energy['Country'] == 'United Kingdom of Great Britain and Northern Ireland19'].Country = 'United Kingdom'
    #energy[energy['Country'] == 'China, Hong Kong Special Administrative Region3'].Country = 'Hong Kong'
    
    energy = energy.replace(['Republic of Korea', 'United States of America20', 'United Kingdom of Great Britain and Northern Ireland19', 'China, Hong Kong Special Administrative Region3'], 
                            ['South Korea', 'United States', 'United Kingdom', 'Hong Kong'])

    # Removing numbers from country names
    energy['Country'] = energy['Country'].str.replace('\d+', '')


    # Removing Paranthesis from country names
    energy['Country'] = energy['Country'].str.replace(r"\(.*\)","")
    energy['Country'] = energy['Country'].str.strip()
    # Energy Countries
    energy_countries = energy['Country']
    energy.head()


    # Cleaning GDP Data
    gdp = pd.read_csv("world_bank.csv")
    gdp = gdp[4:]
    gdp = gdp.rename(columns={'Data Source' : 'Country'})
    # Replace some country names
    gdp[gdp['Country'] == 'Korea, Rep.'].Country = 'South Korea'
    gdp[gdp['Country'] == 'Iran, Islamic Rep.'].Country = 'Iran'
    gdp[gdp['Country'] == 'Hong Kong SAR, China'].Country = 'Hong Kong'
    
    gdp = gdp.replace(['Korea, Rep.', 'Iran, Islamic Rep.', 'Hong Kong SAR, China'], 
                            ['South Korea', 'Iran', 'Hong Kong'])


    
    # GDP Countries
    gdp_countries = gdp['Country']
    

    gdp = gdp[['Country', 'Unnamed: 49'
              , 'Unnamed: 50', 'Unnamed: 51'
              , 'Unnamed: 52', 'Unnamed: 53'
              , 'Unnamed: 54', 'Unnamed: 55'
              , 'Unnamed: 56', 'Unnamed: 57'
              , 'Unnamed: 58']]

    # ScimEm Cleaning
    ScimEn = pd.read_excel("scimagojr-3.xlsx", header=None, footer=None)
    ScimEn.columns = ScimEn.iloc[0]
    ScimEn = ScimEn.reindex(ScimEn.index.drop(0))
    ScimEn = ScimEn[:15]
    # ScimEn Countries
    ScimEn_countries = ScimEn['Country']

    # Joining DataFrames
    df = energy.merge(ScimEn[['Country', 'Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index']], on='Country', how='right')
    df.head()
    gdp = gdp.rename(columns={'Country' : 'Country',
                            'Unnamed: 49' : '2006', 
                           'Unnamed: 50' : '2007',
                           'Unnamed: 51' : '2008',
                           'Unnamed: 52' : '2009',
                           'Unnamed: 53' : '2010',
                           'Unnamed: 54' : '2011',
                           'Unnamed: 55' : '2012',
                           'Unnamed: 56' : '2013',
                           'Unnamed: 57' : '2014',
                           'Unnamed: 58' : '2015'})

    gdp = gdp[['Country', '2006',
                     '2007', '2008',
                     '2009', '2010',
                     '2011', '2012',
                     '2013', '2014',
                     '2015']]

    df = df.merge(gdp, on='Country', how='left')

    df = df.set_index('Country')
    shape = df.shape
    if ((shape[0] == 15) & (shape[1] == 20)):
        return df
    return None

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [175]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [176]:
def answer_two():
    length = len(energy_countries) + len(gdp_countries) + len(ScimEn_countries)
    set_energy = set(energy_countries)
    set_gdp = set(gdp_countries)
    set_scimen = set(ScimEn_countries)
    intersect = set_energy.intersection(set_gdp)
    intersect = intersect.intersection(ScimEn_countries)
    difference = length - len(intersect)
    return difference

<br>

Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [178]:
def answer_three():
    Top15 = answer_one()
    avgGDP = Top15[['2006', '2007','2008', '2009',
                    '2010', '2011', '2012', '2013',
                    '2014', '2015']].mean(axis=1)
    
    return avgGDP

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [181]:
def answer_four():
    Top15 = answer_one()
    sixth_largest = Top15[Top15['Rank'] == 6]
    change = int(sixth_largest['2015']) - int(sixth_largest['2006'])
    return change

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [185]:
def answer_five():
    Top15 = answer_one()
    answer = Top15['Energy Supply per Capita'].mean()
    return answer

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [214]:
def answer_six():
    Top15 = answer_one()
    maximum = Top15['% Renewable'].max()
    country = str(Top15['% Renewable'].idxmax())
    return (country, maximum)

('Brazil', 69.648030000000006)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [215]:
def answer_seven():
    Top15 = answer_one()
    Top15['Ratio'] = Top15['Self-citations'] / Top15['Citations per document']
    maximum = Top15['Ratio'].max()
    country = str(Top15['Ratio'].idxmax())
    return (country, maximum)

('China', 87592.12765957447)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [217]:
def answer_eight():
    Top15 = answer_one()
    Top15['Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    maximum = Top15['Population'].max()
    country = str(Top15['Population'].idxmax())
    return (country, maximum)

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [227]:
def answer_nine():
    Top15 = answer_one()
    population = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable Per Person'] = Top15['Citable documents'] / population
    correlation = Top15['Citable Per Person'].corr(Top15['Energy Supply per Capita'], method='spearman')
    return correlation

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [234]:
def answer_ten():
    Top15 = answer_one()
    median = Top15['% Renewable'].median()
    HighRenew = Top15[Top15['% Renewable'] > median]
    return HighRenew.sort_values(by=['Rank'])

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [250]:
def answer_eleven():
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    Top15 = answer_one().reset_index()
    Top15['Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Country'] = Top15['Country'].map(ContinentDict)
    Top15 = Top15.groupby('Country', as_index=True).agg({
       'Population': ['count', np.sum, np.mean, np.std]})
    return Top15
answer_eleven()

Population                                          
                   count           sum          mean           std
Country                                                           
Asia                   5  2.898666e+09  5.797333e+08  6.790979e+08
Australia              1  2.331602e+07  2.331602e+07           NaN
Europe                 6  4.579297e+08  7.632161e+07  3.464767e+07
North America          2  3.528552e+08  1.764276e+08  1.996696e+08
South America          1  2.059153e+08  2.059153e+08           NaN

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [252]:
def answer_twelve():
    Top15 = answer_one()
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    Top15 = answer_one().reset_index()
    Top15['Population'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Country'] = Top15['Country'].map(ContinentDict)
    Top15['bins'] = pd.cut(Top15['% Renewable'],5)
    return Top15.groupby(['Country', 'bins']).size()

Country        bins            
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
dtype: int64

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [260]:
def answer_thirteen():
    Top15 = answer_one()
    Top15['PopEst'] = (Top15['Energy Supply'] / Top15['Energy Supply per Capita']).astype(float)
    return Top15['PopEst'].apply(lambda x: '{0:,}'.format(x))